# Document Search, Clustering and Classification

## Section 1 - Exploring nltk corpora

The *nltk.corpus* module provides APIs for reading many corpora, including predefined ones downloaded using *ntlk.download()*. One of them is the gutenberg corpus containing about a dozen books.

In [2]:
import nltk
from nltk.corpus import gutenberg

# fileids() tells us the file names in this corpus...
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

<br/>
<br/>
Use **raw()** to get the raw text...

In [3]:
# raw() without arguments reads *all* the files in the corpus
content = gutenberg.raw('austen-emma.txt')
content[:50]

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\n'

<br/>
<br/>
## Tokenization
Use **words()** to read the file and get the list of tokens...

In [4]:
words = gutenberg.words('austen-emma.txt')
print(words[:10])

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER']


<br/>
<br/>
Or **word_tokenize()** if raw text content is already available...

In [5]:
words2 = nltk.word_tokenize(content)
words2[:10]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER']

<br/>
<br/>
Or its variant **wordpunct_tokenize()** which behaves like word_tokenize() but also treats all punctuation marks as separators...

In [6]:
words3 = nltk.wordpunct_tokenize('I don\'t')
words3[:10]

['I', 'don', "'", 't']

<br/>
<br/>

## Cleaning up the token stream

Notice that **words()** has the following problems: 

- Marks like '[' and ']' are tokens, which while not incorrect, does not help us when it comes to document search or classification

- Same words with different cases are treated as different words

In [7]:
# Check if there are any words without any alphanumeric characters.
non_alphanumeric = [w for w in words if not w.isalnum()]
print(len(non_alphanumeric))
print(non_alphanumeric[:10])

# Check if words with different cases are treated as different words.
print(sum(1 for w in words if w=='The'))
print(sum(1 for w in words if w=='the'))

30815
['[', ']', ',', ',', ',', ',', ',', ';', '-', '.']
357
4844


<br/>
<br/>
We'll now clean up this token stream...

In [8]:
cleaned_up = [w.lower() for w in words if w.isalnum()]
print(len(words))
print(len(cleaned_up))
print(cleaned_up[:10])

192427
161612
['emma', 'by', 'jane', 'austen', '1816', 'volume', 'i', 'chapter', 'i', 'emma']


<br/>
<br/>
## Stop word removal

*Stop words* are filler words like 'a', 'an', 'the', 'I', ... which in this use case are just noise for our tasks like search and classification, and can be removed.

Every language has its own set of stop words, and nltk supports most European languages. We load the English stopwords set and list it...

In [9]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
print(stop_words,)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

Remove stop words from our tokens...

In [10]:
filtered = [w for w in cleaned_up if w not in stop_words]
print(len(filtered))
print(filtered[:10])

73198
['emma', 'jane', 'austen', '1816', 'volume', 'chapter', 'emma', 'woodhouse', 'handsome', 'clever']


<br/>
<br/>
But does it cover all stop words? Let's check by printing out the most frequent words in this filtered list...

In [11]:
from collections import Counter
print(Counter(filtered).most_common(30),)

[('mr', 1153), ('emma', 865), ('could', 837), ('would', 820), ('mrs', 699), ('miss', 599), ('must', 567), ('harriet', 506), ('much', 486), ('said', 484), ('one', 452), ('weston', 440), ('every', 435), ('well', 401), ('thing', 398), ('knightley', 389), ('elton', 385), ('think', 383), ('little', 359), ('good', 358), ('never', 358), ('know', 337), ('might', 326), ('woodhouse', 313), ('say', 310), ('jane', 301), ('quite', 282), ('time', 279), ('great', 264), ('nothing', 256)]


<br/>
<br/>
We seee that some filler words like 'would', 'could', 'said', 'might', 'much'... still occur frequently.

Other words like 'mr', 'mrs', 'miss' .... are also borderline stopwords, but we'll retain them for this experiment.

In [12]:
custom_stopwords = ['could', 'would', 'must', 'will', 'much', 'might', 'shall', 'should']
filtered = [w for w in filtered if w not in custom_stopwords]
print(len(filtered))
print(filtered[:10])
from collections import Counter
print(Counter(filtered).most_common(30),)

69945
['emma', 'jane', 'austen', '1816', 'volume', 'chapter', 'emma', 'woodhouse', 'handsome', 'clever']
[('mr', 1153), ('emma', 865), ('mrs', 699), ('miss', 599), ('harriet', 506), ('said', 484), ('one', 452), ('weston', 440), ('every', 435), ('well', 401), ('thing', 398), ('knightley', 389), ('elton', 385), ('think', 383), ('little', 359), ('good', 358), ('never', 358), ('know', 337), ('woodhouse', 313), ('say', 310), ('jane', 301), ('quite', 282), ('time', 279), ('great', 264), ('nothing', 256), ('dear', 241), ('fairfax', 241), ('always', 238), ('man', 235), ('thought', 226)]


<br/>
<br/>
## Word counts / Term Frequencies

We can either implement our own word counter...

In [13]:
from collections import Counter

word_counter = Counter(filtered)
print(word_counter.most_common(30),)




[('mr', 1153), ('emma', 865), ('mrs', 699), ('miss', 599), ('harriet', 506), ('said', 484), ('one', 452), ('weston', 440), ('every', 435), ('well', 401), ('thing', 398), ('knightley', 389), ('elton', 385), ('think', 383), ('little', 359), ('good', 358), ('never', 358), ('know', 337), ('woodhouse', 313), ('say', 310), ('jane', 301), ('quite', 282), ('time', 279), ('great', 264), ('nothing', 256), ('dear', 241), ('fairfax', 241), ('always', 238), ('man', 235), ('thought', 226)]


<br/>
...or use nltk's **FreqDist** class which is in fact a subclass of Counter...

In [14]:
freqs = nltk.FreqDist(filtered)
print(freqs.most_common(30),)

[('mr', 1153), ('emma', 865), ('mrs', 699), ('miss', 599), ('harriet', 506), ('said', 484), ('one', 452), ('weston', 440), ('every', 435), ('well', 401), ('thing', 398), ('knightley', 389), ('elton', 385), ('think', 383), ('little', 359), ('good', 358), ('never', 358), ('know', 337), ('woodhouse', 313), ('say', 310), ('jane', 301), ('quite', 282), ('time', 279), ('great', 264), ('nothing', 256), ('dear', 241), ('fairfax', 241), ('always', 238), ('man', 235), ('thought', 226)]


<br/>
<br/>
## Combine everything done so far into a reusable preprocessing function

For document search or classification, we need to execute all these steps like deduplication and stop word removal  on every document in a corpus. Combine it all into a function that takes a document as input and returns terms and their frequencies...

In [40]:
def term_frequencies(doc):
    words = nltk.word_tokenize(doc)
    cleaned_up = [w.lower() for w in words if w.isalnum()]
    filtered = cleaned_up
    filtered = [w for w in cleaned_up if w not in stop_words]
    filtered = [w for w in filtered if w not in custom_stopwords]
    freqs = nltk.FreqDist(filtered)
    return freqs

# test it...
freqs = term_frequencies(content)
print(freqs.most_common(30),)

[('emma', 855), ('miss', 597), ('harriet', 496), ('said', 483), ('one', 447), ('every', 434), ('weston', 430), ('thing', 394), ('think', 383), ('elton', 378), ('well', 375), ('knightley', 373), ('little', 359), ('never', 358), ('know', 335), ('good', 313), ('say', 310), ('woodhouse', 308), ('jane', 299), ('quite', 282), ('time', 275), ('great', 263), ('nothing', 252), ('dear', 241), ('always', 238), ('man', 232), ('fairfax', 232), ('thought', 225), ('soon', 223), ('see', 222)]


<br/>
<br/>

-----

<br/>

## Section 2 - Understanding TF-IDF

In this section, we understand the concept of TF-IDF, why it's necessary and how to calculate it.

But before that, let's understand the problems themselves - namely document searching, clustering and classification.

<br/>
<br/>

### Understanding the Document Search problem

We have a set of documents. Given a search query, we have to find documents that "match" the query. Additionally, the user expects the query results to be relevant as well as sorted with the most relevant search results at the top.
How can this problem be solved?

Let's take an example...

- Search query: *"book on quantum physics"*

- Document #1: *"A is a good book on physics. B is another good book on physics. C is a good book on algorithms."*

- Document #2: *"D is a well written book and covers quantum physics in detail."*

- Document #3: *"E is a great course on quantum physics."*


Clearly, #2 is the most relevant. But if we use just the word counts and score each document based on number of occurrences of search terms in it, then #1 will score higher simply because the word 'book' occurs more often.

We need a different scoring that penalizes too many occurrences of common words while rewarding few occurrences of rare words. This is where TF-IDF helps...



### What is TF-IDF? And how does it help?

**TF** stands for **Term Frequences** which we have already calculated previously.

**IDF** is **Inverse Document Frequency**. Every *term* has an IDF score. Conceptually, IDF is a measure of the ratio of (total number of documents in the corpus) to (number of documents that contain that term). 
Mathematically, it's defined as 
$$IDF_{term} = log \frac{numdocs_{total}}{1 + numdocs_{term}}$$

We can see that if $numdocs_{term} \approx numdocs_{total}$, IDF becomes log 1 = 0. So a term that is so common that it occurs in (nearly) every document in the corpus gets a very low, near-zero IDF weight.

Conversely, a rare term that occurs in only a few documents, such that $numdocs_{term} \lt\lt numdocs_{total}$, makes its IDF score close to its maximum.

Note that if a term does not occur at all in any document, then the IDF is maximum which is silly. The 1 in denominator is to avoid division by zero in case a term does not occur in any document. Such a term should simply be ignored, since its term frequency and IDF will be zero in all documents.

The **TF-IDF score** of a term in a document is the product of its (TF in that document) and (IDF across documnents).

Instead of describing a document as a vector of word counts, we describe it as a vector of tf-idf scores containing a score for every term across the entire corpus.

In [41]:
all_terms = {}
doc_freqs = {}
for f in gutenberg.fileids():
    doc = gutenberg.raw(f)
    freqs = term_frequencies(doc)
    doc_freqs[f] = freqs
    print(len(freqs))
    for term in freqs.keys():
        if term in all_terms:
            all_terms[term] += 1
        else:
            all_terms[term] = 1
            

6809
5561
6137
12434
1378
3622
1346
2340
7914
7387
6052
8055
16561
8751
2836
4452
3204
11879


In [49]:
len(all_terms)

40897

In [42]:
import math

idf_scores = {}
for t in list(all_terms):
    num_docs_with_term = all_terms[t]
    idf_scores[t] = math.log10(N / (1 + num_docs_with_term))
    
for t in list(idf_scores)[:10]:
    print(t, all_terms[t], idf_scores[t])

tier 2 0.7781512503836436
stephanas 1 0.9542425094393249
ahijah 1 0.9542425094393249
recrossing 1 0.9542425094393249
searches 2 0.7781512503836436
senegal 1 0.9542425094393249
suppliant 1 0.9542425094393249
dallying 2 0.7781512503836436
cowardice 3 0.6532125137753437
methegammah 1 0.9542425094393249


In [43]:
import operator

sorted_scores = sorted(idf_scores.items(), key=operator.itemgetter(1))
print(sorted_scores[:5])
print(sorted_scores[-6:-1])

[('know', -0.023481095849522914), ('heard', -0.023481095849522914), ('saying', -0.023481095849522914), ('day', -0.023481095849522914), ('found', -0.023481095849522914)]
[('biblic', 0.9542425094393249), ('intercedings', 0.9542425094393249), ('wineglasses', 0.9542425094393249), ('demetrius', 0.9542425094393249), ('eleve', 0.9542425094393249)]


In [45]:
tf_idf_vectors = {}
least_tf_idf = (None, 0.0, 0.0, 0.0, None)
highest_tf_idf = (None, 0.0, 0.0, 0.0)
for f in list(doc_freqs):
    term_freqs = doc_freqs[f]
    tf_idf_vectors[f] = {}
    for t in list(term_freqs):
        tf_idf_score_for_term = term_freqs[t] * idf_scores[t]
        tf_idf_vectors[f][t] = (tf_idf_score_for_term, term_freqs[t], idf_scores[t])
        
        if tf_idf_score_for_term < least_tf_idf[1]:
            least_tf_idf = (t, tf_idf_score_for_term, term_freqs[t], idf_scores[t], f)
            
        if tf_idf_score_for_term > highest_tf_idf[1]:
            highest_tf_idf = (t, tf_idf_score_for_term, term_freqs[t], idf_scores[t], f)
        
        
for f,tf_idf_vector in tf_idf_vectors.items():
    print(f)
    for t in list(tf_idf_vector)[:10]:
        print(t, tf_idf_vector[t])
    print()

austen-emma.txt
decree (0.4101744650890493, 1, 0.4101744650890493)
stop (0.0, 15, 0.0)
reproach (3.1696426630022625, 9, 0.3521825181113625)
look (-2.794250406093227, 119, -0.023481095849522914)
military (0.25527250510330607, 1, 0.25527250510330607)
damps (0.7781512503836436, 1, 0.7781512503836436)
strictly (0.5105450102066121, 2, 0.25527250510330607)
walk (0.0, 53, 0.0)
parlours (1.5563025007672873, 2, 0.7781512503836436)
consternation (0.8203489301780986, 2, 0.4101744650890493)

whitman-leaves.txt
tier (1.5563025007672873, 2, 0.7781512503836436)
traitors (1.0565475543340874, 3, 0.3521825181113625)
senegal (0.9542425094393249, 1, 0.9542425094393249)
dallying (3.1126050015345745, 4, 0.7781512503836436)
strictly (0.25527250510330607, 1, 0.25527250510330607)
walk (0.0, 66, 0.0)
florida (5.447058752685505, 7, 0.7781512503836436)
laughingly (0.9542425094393249, 1, 0.9542425094393249)
coasts (2.225210003069149, 4, 0.5563025007672873)
airs (2.4082399653118496, 8, 0.3010299956639812)

melville

In [46]:
least_tf_idf

('said', -93.90090230224213, 3999, -0.023481095849522914, 'bible-kjv.txt')

In [47]:
highest_tf_idf

('unto', 3690.3396624061766, 8997, 0.4101744650890493, 'bible-kjv.txt')